## LIN350 Course Project - The Language of Immigration Politics: Terminology Differences Across Party Lines in Congressional Speeches

The way I usually run jupyter notebooks is opening the anaconda prompt terminal and running the command *jupyter notebook* from there I go to visual studio and click on select kernel -> existing jupyter server -> localhost or you can copy and paste the url of the tab that opened up with the *jupyter notebook* command and then click on python and that should be it

To keep track of the work we're doing together we can use a github repository to update changes and sync up our work. The usual workflow for this should be.
1. Any changes you have in your laptop can be added to the repository with "git add ./" from the terminal the notebook is in
2. After adding the files and changes you can use "git commit -m 'message here'" For the message make sure its in quotations and it can be anything
3. After adding and commiting you can "git push" which pushes ur changes to the repository
4. Let's say there's changes in the repository that are not in your laptop you can fetch them with "git pull"

Some other setup you might need to do is set environement variables in local computer since we don't want to share that in the repository for privacy issues. So to do this you would run commands in your notebook to set it up. I'll show you
1. running "%env" in a code block will show you all the environment variables in the jupyter environment
2. to set up the enviroment variable for our project run the command "%env API_KEY=apikeyfromourgoogledocs"
3. After that running the first cell of code will setup the api key to be used as API_KEY


In [ ]:
# 

env: API_KEY=qAyZUrTJs4fdKIPefgekdiMQrCchdt979fIo58M1


### Congressional Record Data Collector - Very simple for now, simple text data collection

In [ ]:
# !pip install Xlsxwriter

In [ ]:
import requests
import json
import os
import pandas as pd
import time
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import glob
import re
from bs4 import BeautifulSoup
from tqdm import tqdm  # For progress bars
import xlsxwriter

# create directories for data storage
os.makedirs('data/expanded_data_now', exist_ok=True)

# set your API key here (get one from https://api.data.gov/signup/)
API_KEY = os.environ["API_KEY"]

# define date ranges for your study (immigration debates 2018-2023)
date_ranges = [
    # 2019 - Border wall government shutdown -> for now 
    ("2019-01-01", "2019-01-31"),
    
    # Government shutdown over border wall funding
    ("2018-12-15", "2018-12-31"),

    # DACA debates
    ("2017-09-01", "2017-10-15"),
    ("2018-01-15", "2018-02-15"),
    
    # Border surge discussions
    ("2019-03-01", "2019-04-15"),
    
    # Election year immigration discussions
    ("2020-01-15", "2020-02-15"),
    ("2020-09-01", "2020-10-15"),
    
    # Biden administration policy changes
    ("2021-01-20", "2021-03-01")
]
# immigration-related terms with more precise matching
immigration_terms = {
    # regular terms - can appear within other words
    'immigration': r'immigration',
    'immigrant': r'immigrant',
    'migrant': r'migrant',
    'citizenship': r'citizenship',
    'deportation': r'deportation',
    
    # terms that need word boundary checks
    'border': r'\b(?:border|borders)\b',
    'asylum': r'\basylum\b',
    'refugee': r'\b(?:refugee|refugees)\b',
    'undocumented': r'\bundocumented\b',
    'illegal alien': r'\billegal\s+alien',
    'unauthorized': r'\bunauthorized\b',
    'wall': r'\bwall\b',
    'daca': r'\bdaca\b',
    'dreamer': r'\b(?:dreamer|dreamers)\b',
    'visa': r'\bvisa\b',
    'detention': r'\bdetention\b',
    
    # phrases
    'family separation': r'family\s+separation',
    'child detention': r'child\s+detention',
    'border security': r'border\s+security',
    'border crisis': r'border\s+crisis',
    'path to citizenship': r'path\s+to\s+citizenship',
    'amnesty': r'\bamnesty\b',
    'caravan': r'\bcaravan\b',
    
    # specific entities
    'mexico': r'\bmexico\b',
    'ice': r'\b(?:ice|immigration and customs enforcement)\b',  # Only match whole word "ice"
    'cbp': r'\b(?:cbp|customs and border protection)\b'
}

In [6]:
"""Generate all dates in a given range, essentially given range makes a list of day dates"""
def get_dates_in_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    date_list = []
    current = start
    while current <= end:
        date_list.append(current.strftime("%Y-%m-%d"))
        current += timedelta(days=1)
    return date_list

In [7]:
"""
Function to get Congressional Record data using the GovInfo API (this worked thankfully)
Args:
    date (str): Date in YYYY-MM-DD format
Returns:
    bool: Success status
"""
def get_congressional_record(date):
   
    package_id = f"CREC-{date}"
    
    # first check if the package exists for this date 
    package_url = f"https://api.govinfo.gov/packages/{package_id}/summary"
    
    params = {
        'api_key': API_KEY
    }
    
    try:
        # check if the package exists
        response = requests.get(package_url, params=params)
        
        # if package doesn't exist or other error
        if response.status_code != 200:
            print(f"No Congressional Record available for {date} (Status: {response.status_code})")
            return False
        
        # save the package summary
        with open(f"data/congressional_record/{package_id}-summary.json", 'w') as f:
            json.dump(response.json(), f)
        
        # get granules (speeches and entries) 
        granules_url = f"https://api.govinfo.gov/packages/{package_id}/granules"
        granules_params = {
            'api_key': API_KEY,
            'offset': 0,
            'pageSize': 100  # Max page size
        }
        
        # get first page of granules
        granules_response = requests.get(granules_url, params=granules_params)
        
        if granules_response.status_code != 200:
            print(f"Failed to get granules for {date} (Status: {granules_response.status_code})")
            return False
            
        # save the granules list
        with open(f"data/congressional_record/{package_id}-granules.json", 'w') as f:
            json.dump(granules_response.json(), f)
            
        # download content for each granule
        granules = granules_response.json().get('granules', [])
        
        for granule in granules:
            granule_id = granule.get('granuleId')
            
            # skip if no granule ID
            if not granule_id:
                continue
            
            # get the HTML content
            content_url = f"https://api.govinfo.gov/packages/{package_id}/granules/{granule_id}/htm"
            content_response = requests.get(content_url, params=params)
            
            if content_response.status_code == 200:
                # save the HTML content
                with open(f"data/congressional_record/{package_id}-{granule_id}.html", 'w', encoding='utf-8') as f:
                    f.write(content_response.text)
            
            # respect rate limits
            time.sleep(0.5)
            
        print(f"Successfully downloaded Congressional Record for {date} ({len(granules)} granules)")
        return True
        
    except Exception as e:
        print(f"Error retrieving data for {date}: {e}")
        return False

In [8]:
# the following is just an api key verification to make sure it works
import requests

# The API key you provided
API_KEY = "qAyZUrTJs4fdKIPefgekdiMQrCchdt979fIo58M1"

# Test the API key with a simple request to the GovInfo API
test_url = "https://api.govinfo.gov/collections"
params = {
    'api_key': API_KEY
}

try:
    print("Testing API key with GovInfo API...")
    response = requests.get(test_url, params=params)
    
    if response.status_code == 200:
        print("✅ Success! Your API key is valid for the GovInfo API.")
        print(f"Status code: {response.status_code}")
        
        # Show the first few collections to confirm we got real data
        collections = response.json().get('collections', [])
        if collections:
            print("\nAvailable collections:")
            for collection in collections[:5]:
                print(f"- {collection.get('collectionName', 'Unknown')}")
    elif response.status_code == 401 or response.status_code == 403:
        print("❌ Authentication failed. Your API key appears to be invalid.")
        print(f"Status code: {response.status_code}")
        print(f"Response: {response.text}")
    else:
        print(f"⚠️ Received unexpected status code: {response.status_code}")
        print(f"Response: {response.text}")
        
except Exception as e:
    print(f"❌ Error occurred while testing the API key: {e}")

Testing API key with GovInfo API...
✅ Success! Your API key is valid for the GovInfo API.
Status code: 200

Available collections:
- Congressional Bills
- Congressional Bill Status
- Congressional Bill Summaries
- United States Budget
- Congressional Calendars


In [24]:
"""
Main function to download Congressional Record data
"""
def main():
   
    all_dates = []
    
    # generate all dates in the specified ranges
    for start_date, end_date in date_ranges:
        dates = get_dates_in_range(start_date, end_date)
        all_dates.extend(dates)
    
    print(f"Will download Congressional Record data for {len(all_dates)} dates")
    
    # download data for each date
    successful_downloads = 0
    for date in tqdm(all_dates):
        success = get_congressional_record(date)
        if success:
            successful_downloads += 1
        
        # wait between requests to avoid rate limiting
        time.sleep(1)
    
    print(f"\nData collection complete!")
    print(f"Successfully downloaded data for {successful_downloads} out of {len(all_dates)} dates")
    print(f"Data saved to: data/congressional_record/")
    
    # create a simple summary file with immigration keywords to help with later analysis
    with open('data/immigration_keywords.txt', 'w') as f:
        keywords = [
            'immigration', 'immigrant', 'migrant', 'migration', 'asylum', 
            'refugee', 'border', 'wall', 'undocumented', 'illegal alien',
            'daca', 'dreamer', 'deportation', 'visa', 'citizenship',
            'family separation', 'child detention', 'border security',
            'border crisis', 'caravan', 'amnesty', 'path to citizenship'
        ]
        f.write('\n'.join(keywords))

if __name__ == "__main__":
    main()

Will download Congressional Record data for 289 dates


  0%|          | 0/289 [00:00<?, ?it/s]

No Congressional Record available for 2019-01-01 (Status: 404)
Successfully downloaded Congressional Record for 2019-01-02 (72 granules)
No Congressional Record available for 2019-01-03 (Status: 404)
Successfully downloaded Congressional Record for 2019-01-04 (100 granules)
No Congressional Record available for 2019-01-05 (Status: 404)
No Congressional Record available for 2019-01-06 (Status: 404)
No Congressional Record available for 2019-01-07 (Status: 404)
Successfully downloaded Congressional Record for 2019-01-08 (100 granules)
Successfully downloaded Congressional Record for 2019-01-09 (100 granules)
Successfully downloaded Congressional Record for 2019-01-10 (100 granules)
Successfully downloaded Congressional Record for 2019-01-11 (100 granules)
No Congressional Record available for 2019-01-12 (Status: 404)
No Congressional Record available for 2019-01-13 (Status: 404)
Successfully downloaded Congressional Record for 2019-01-14 (100 granules)
Successfully downloaded Congression

In [9]:
# I just wanted to see if there were actually any immigrant related words in the data I collected so far, thankfully there was so now I want to convert these html files into
# csv/excel files to have it more localized
path = r"C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\congressional_record"
html_files = glob.glob(os.path.join(path, "*.html"))

# Search ALL files (this might take a few minutes)
immigration_files = []
for file in html_files:
    with open(file, 'r', encoding='utf-8') as f:
        try:
            content = f.read().lower()
            found_terms = [term for term in immigration_terms if term in content]
            if found_terms:
                immigration_files.append({
                    'file': file,
                    'terms': found_terms
                })
        except Exception as e:
            print(f"Error reading {file}: {e}")

print(f"Found {len(immigration_files)} files with immigration content")

# If we found any, save the list for reference
if immigration_files:
    df = pd.DataFrame(immigration_files)
    df.to_csv(os.path.join(path, "immigration_files.csv"), index=False)
    print("First 5 files with immigration content:")
    for item in immigration_files[:5]:
        print(f"  - {os.path.basename(item['file'])}: {', '.join(item['terms'])}")

Found 0 files with immigration content


In [10]:
# directory setup
data_dir = r"C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\congressional_record\congressional_record"
output_dir = r"C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data"
os.makedirs(output_dir, exist_ok=True)

print("Libraries imported successfully!")
print(f"Input directory: {data_dir}")
print(f"Output directory: {output_dir}")

Libraries imported successfully!
Input directory: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\congressional_record\congressional_record
Output directory: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data


In [9]:
# get list of all HTML files
html_files = glob.glob(os.path.join(data_dir, "*.html"))
total_files = len(html_files)

print(f"Found {total_files} HTML files in {data_dir}")

# check the first few files to make sure we can access them
if total_files > 0:
    print("\nSample filenames:")
    for file in html_files[:5]:
        print(f"  - {os.path.basename(file)}")
    
    # try to open one file to verify access
    try:
        with open(html_files[0], 'r', encoding='utf-8') as f:
            first_chars = f.read(200)
        print("\nSuccessfully read first file. First 200 characters:")
        print(first_chars.replace('\n', ' ')[:200])
    except Exception as e:
        print(f"\nError reading file: {e}")

Found 14629 HTML files in C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\congressional_record\congressional_record

Sample filenames:
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-2.html
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-3.html
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-4.html
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-5.html
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-6.html

Successfully read first file. First 200 characters:
<html> <head> <title>Congressional Record, Volume 163 Issue 141 (Friday, September 1, 2017)</title> </head> <body><pre> [Congressional Record Volume 163, Number 141 (Friday, September 1, 2017)] [Daily


In [13]:
# search for immigration-related content
immigration_files = []
print(f"Searching {total_files} files for immigration content...")

for file in tqdm(html_files):
    try:
        with open(file, 'r', encoding='utf-8') as f:
            content = f.read().lower()
            
        # Check each term with its specific regex pattern
        found_terms = []
        for term, pattern in immigration_terms.items():
            if re.search(pattern, content):
                found_terms.append(term)
        
        if found_terms:
            # Extract date from filename
            filename = os.path.basename(file)
            date_parts = filename.split('-')
            if len(date_parts) >= 2:
                date = date_parts[1]
            else:
                date = "Unknown"
            
            immigration_files.append({
                'file': file,
                'date': date,
                'terms': ', '.join(found_terms)  # Convert list to string
            })
    except Exception as e:
        print(f"Error processing {os.path.basename(file)}: {e}")

# Save results to CSV
if immigration_files:
    immigration_df = pd.DataFrame(immigration_files)
    csv_path = os.path.join(output_dir, "immigration_files.csv")
    immigration_df.to_csv(csv_path, index=False)
    print(f"\nFound {len(immigration_files)} files with immigration content")
    print(f"List saved to: {csv_path}")
    
    # Show sample of found files
    print("\nSample immigration-related files:")
    for file_info in immigration_files[:5]:
        print(f"  - {os.path.basename(file_info['file'])}: {file_info['terms']}")
else:
    print("No immigration-related files found.")

Searching 14629 files for immigration content...


100%|███████████████████████████████████████████████████████████████████████████| 14629/14629 [00:30<00:00, 485.38it/s]


Found 1785 files with immigration content
List saved to: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data\immigration_files.csv

Sample immigration-related files:
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-6.html: visa
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgE1151-4.html: refugee
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgE1152-3.html: immigration, immigrant, migrant, citizenship, deportation, undocumented, daca, dreamer, visa
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgE1154-4.html: undocumented, mexico
  - CREC-2017-09-01-CREC-2017-09-01-pt1-PgH6632-6.html: mexico


In [16]:
immigration_files_csv = os.path.join(output_dir, "immigration_files.csv")
# Load the list of immigration-related files
if not os.path.exists(immigration_files_csv):
    print(f"Error: Immigration files list not found at {immigration_files_csv}")
    exit()

immigration_df = pd.read_csv(immigration_files_csv)
print(f"Processing {len(immigration_df)} immigration-related files...")

# Function to extract structured data from HTML
def parse_congressional_record(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Parse HTML
        soup = BeautifulSoup(content, 'html.parser')
        pre_content = soup.find('pre')
        
        if not pre_content:
            return None
        
        text = pre_content.get_text()
        
        # Extract date from header
        date_match = re.search(r'\[Congressional Record Volume \d+, Number \d+ \(([^)]+)\)\]', text)
        date = date_match.group(1) if date_match else "Unknown"
        
        # Extract chamber (Senate or House)
        chamber = "Unknown"
        if "Senate" in text[:500]:
            chamber = "Senate"
        elif "House" in text[:500]:
            chamber = "House"
        
        # Extract speaker - look for common patterns
        speaker_patterns = [
            # Pattern for "Mr. SMITH" format
            r'(?:^|\n)\s{1,10}((?:Mr\.|Mrs\.|Ms\.|Senator|Representative)\s+([A-Z]+))\.', 
            # Pattern for "The CHAIRMAN" format
            r'(?:^|\n)\s{1,10}(The\s+([A-Z]+))\.', 
            # Fallback pattern for other formats
            r'(?:^|\n)\s{1,10}([A-Z]{2,}(?:\s+[A-Z]+)*)\.?' 
        ]
        
        speaker_full = "Unknown"
        speaker_last = "Unknown"
        
        for pattern in speaker_patterns:
            match = re.search(pattern, text)
            if match:
                speaker_full = match.group(1)
                if len(match.groups()) > 1 and match.group(2):
                    speaker_last = match.group(2)
                else:
                    # Extract last name from full name
                    parts = speaker_full.split()
                    if parts:
                        speaker_last = parts[-1]
                break
        
        # Extract title - look for all-caps line after header
        title_match = re.search(r'\n\s*([A-Z][A-Z\s\'\",.()-]+?)\s*\n', text)
        title = title_match.group(1).strip() if title_match else "Unknown"
        
        # Get granule ID from filename
        filename = os.path.basename(file_path)
        granule_id = filename.replace(".html", "")
        
        return {
            'file_id': granule_id,
            'date': date,
            'chamber': chamber,
            'speaker_full': speaker_full,
            'speaker_last': speaker_last,
            'title': title,
            'full_text': text
        }
    except Exception as e:
        print(f"Error parsing {os.path.basename(file_path)}: {e}")
        return None

# Process each file in the immigration list
parsed_data = []
for _, row in tqdm(immigration_df.iterrows(), total=len(immigration_df)):
    file_path = row['file']
    extracted_data = parse_congressional_record(file_path)
    
    if extracted_data:
        # Add the immigration terms found
        extracted_data['immigration_terms'] = row['terms']
        parsed_data.append(extracted_data)

# Create a DataFrame and save to CSV
if parsed_data:
    parsed_df = pd.DataFrame(parsed_data)
    csv_path = os.path.join(output_dir, "immigration_speeches.csv")
    parsed_df.to_csv(csv_path, index=False)
    
    print(f"\nSuccessfully parsed {len(parsed_data)} files")
    print(f"Data saved to: {csv_path}")
    
    # Print summary of speakers found
    speaker_counts = parsed_df['speaker_last'].value_counts()
    print(f"\nTop 10 speakers in the dataset:")
    print(speaker_counts.head(10))
    
    # Print example of first record
    print("\nExample of parsed data (first record):")
    for key, value in parsed_data[0].items():
        if key == 'full_text':
            print(f"{key}: {value[:200]}...") # Print only first 200 chars of text
        else:
            print(f"{key}: {value}")
else:
    print("No data could be parsed from the files.")

Processing 1785 immigration-related files...


  0%|                                                                                         | 0/1785 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 1785/1785 [00:02<00:00, 723.86it/s]



Successfully parsed 1785 files
Data saved to: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data\immigration_speeches.csv

Top 10 speakers in the dataset:
speaker_last
Unknown       510
RESOLUTION     79
SCHUMER        70
SA             57
CORNYN         40
DURBIN         40
EC             28
FEINSTEIN      25
THUNE          24
GRASSLEY       23
Name: count, dtype: int64

Example of parsed data (first record):
file_id: CREC-2017-09-01-CREC-2017-09-01-pt1-PgD909-6
date: Friday, September 1, 2017
chamber: House
speaker_full: Unknown
speaker_last: Unknown
title: HOUSE
full_text: 
[Congressional Record Volume 163, Number 141 (Friday, September 1, 2017)]
[Daily Digest]
[Pages D909-D910]
From the Congressional Record Online through the Government Publishing Office [www.gpo.gov]
...
immigration_terms: visa


In [11]:
speeches_csv = os.path.join(output_dir, "immigration_speeches.csv")

# Load the existing data
speeches_df = pd.read_csv(speeches_csv)
print(f"Loaded {len(speeches_df)} records from original CSV")

# Function to clean and improve data
def clean_data(df):
    # Create a copy to avoid modifying the original
    cleaned_df = df.copy()
    
    # 1. Convert dates to standard format
    def standardize_date(date_str):
        try:
            if pd.isna(date_str) or date_str == "Unknown":
                return None
            # Parse date string to datetime object
            date_obj = datetime.strptime(date_str, "%A, %B %d, %Y")
            # Convert to standard format
            return date_obj.strftime("%Y-%m-%d")
        except:
            return date_str
    
    cleaned_df['date_standard'] = cleaned_df['date'].apply(standardize_date)
    
    # 2. Identify real speeches vs. procedural text
    def is_real_speech(row):
        # Check if it's likely a speech by a member of Congress
        
        # If speaker is Unknown, probably not a speech
        if row['speaker_last'] == "Unknown":
            return False
        
        # Check for procedural titles
        procedural_titles = ['HOUSE', 'SENATE', 'PRAYER', 'PLEDGE', 'ADJOURNMENT', 
                            'RECESS', 'AMENDMENT', 'RECORD', 'MOTION', 'RESOLUTION']
        if any(title in row['title'] for title in procedural_titles):
            return False
        
        # Check for very short texts (likely not speeches)
        if len(row['full_text']) < 500:
            return False
            
        return True
    
    cleaned_df['is_speech'] = cleaned_df.apply(is_real_speech, axis=1)
    
    # 3. Categorize speech type
    def categorize_speech(row):
        text = row['full_text'].lower()
        
        if not row['is_speech']:
            return "procedural"
            
        categories = {
            "border_security": ["border security", "border wall", "border crisis"],
            "legal_status": ["undocumented", "illegal alien", "unauthorized", "amnesty", "path to citizenship"],
            "children": ["daca", "dreamer", "child", "family separation"],
            "asylum": ["asylum", "refugee", "humanitarian"],
            "general": ["immigration", "immigrant", "migrant"]
        }
        
        for category, terms in categories.items():
            if any(term in text for term in terms):
                return category
                
        return "other"
    
    cleaned_df['speech_category'] = cleaned_df.apply(categorize_speech, axis=1)
    
    # 4. Extract a summary from the full text (first 300 characters)
    def extract_summary(text):
        # Remove header content in square brackets
        text = re.sub(r'\[.*?\]', '', text)
        # Remove whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        # Take first 300 characters
        return text[:300] + "..." if len(text) > 300 else text
    
    cleaned_df['speech_summary'] = cleaned_df['full_text'].apply(extract_summary)
    
    # 5. Add party information for key members
    party_lookup = {
        'MCCONNELL': 'R', 'SCHUMER': 'D', 'DURBIN': 'D', 'THUNE': 'R', 'CORNYN': 'R',
        'GRASSLEY': 'R', 'FEINSTEIN': 'D', 'LEAHY': 'D', 'CRUZ': 'R', 'GRAHAM': 'R',
        'PELOSI': 'D', 'MCCARTHY': 'R', 'HOYER': 'D', 'SCALISE': 'R', 'CLYBURN': 'D',
        'HARRIS': 'D', 'WARREN': 'D', 'SANDERS': 'I', 'RUBIO': 'R', 'COTTON': 'R'
    }
    
    cleaned_df['party'] = cleaned_df['speaker_last'].map(party_lookup)
    
    # 6. Keep only relevant columns in a useful order
    columns_order = [
        'file_id', 'date_standard', 'chamber', 'speaker_full', 'speaker_last', 
        'party', 'title', 'is_speech', 'speech_category', 'speech_summary', 
        'immigration_terms', 'full_text'
    ]
    
    # Return only the columns we want
    return cleaned_df[columns_order]

# Clean the data
cleaned_speeches = clean_data(speeches_df)

# Create filtered dataset with only actual speeches
speeches_only = cleaned_speeches[cleaned_speeches['is_speech'] == True]

# Save the cleaned and filtered data
cleaned_speeches.to_csv(os.path.join(output_dir, "immigration_data_clean.csv"), index=False)
speeches_only.to_csv(os.path.join(output_dir, "immigration_speeches_clean.csv"), index=False)

# Print summary
print(f"\nCreated cleaned dataset with {len(cleaned_speeches)} records")
print(f"Created filtered dataset with {len(speeches_only)} actual speeches")

# Show party distribution 
if len(speeches_only) > 0:
    party_counts = speeches_only['party'].value_counts()
    print("\nParty distribution in speeches:")
    print(party_counts)

# Show category distribution
category_counts = speeches_only['speech_category'].value_counts()
print("\nSpeech category distribution:")
print(category_counts)

Loaded 1785 records from original CSV

Created cleaned dataset with 1785 records
Created filtered dataset with 1094 actual speeches

Party distribution in speeches:
party
D    147
R     92
I     10
Name: count, dtype: int64

Speech category distribution:
speech_category
children           406
other              283
border_security    181
general             92
legal_status        89
asylum              43
Name: count, dtype: int64


In [19]:
import csv

# Directory paths
base_dir = r"C:\Users\kevin barcenas\Documents\GitHub\LIN350Project"
output_dir = os.path.join(base_dir, "processed_data")
speeches_csv = os.path.join(output_dir, "immigration_speeches_clean.csv")

# Load the existing data
speeches_df = pd.read_csv(speeches_csv)
print(f"Loaded {len(speeches_df)} records from clean speeches CSV")

# Check available columns
print("Available columns in the dataset:")
for col in speeches_df.columns:
    print(f"- {col}")

# Create a copy to avoid modifying the original
df_clean = speeches_df.copy()

# Add party information if 'party' column doesn't exist but 'speaker_last' does
if 'speaker_last' in df_clean.columns and 'party' not in df_clean.columns:
    print("\nAdding party information...")
    
    party_lookup = {
        # Senate leadership
        'MCCONNELL': 'R', 'SCHUMER': 'D', 'DURBIN': 'D', 'THUNE': 'R', 'CORNYN': 'R',
        'GRASSLEY': 'R', 'FEINSTEIN': 'D', 'LEAHY': 'D', 'CRUZ': 'R', 'GRAHAM': 'R',
        'SANDERS': 'I', 'HARRIS': 'D', 'WARREN': 'D', 'RUBIO': 'R', 'COTTON': 'R',
        # House leadership
        'PELOSI': 'D', 'MCCARTHY': 'R', 'HOYER': 'D', 'SCALISE': 'R', 'CLYBURN': 'D',
        # Add more as needed
        'CAPUANO': 'D', 'MATSUI': 'D', 'HULTGREN': 'R', 'KRISHNAMOORTHI': 'D',
        'PAYNE': 'D', 'LANGEVIN': 'D', 'BLUMENAUER': 'D', 'ROHRABACHER': 'R'
    }
    
    df_clean['party'] = df_clean['speaker_last'].map(party_lookup)
    df_clean['party'] = df_clean['party'].fillna('Unknown')

# Add speech categorization if needed
if 'full_text' in df_clean.columns and 'speech_category' not in df_clean.columns:
    print("Adding speech categorization...")
    
    def categorize_speech(text):
        if pd.isna(text):
            return "unknown"
            
        text = str(text).lower()
        
        if any(term in text for term in ["border security", "border wall", "border crisis", "wall"]):
            return "border_security"
        elif any(term in text for term in ["undocumented", "illegal alien", "unauthorized", "amnesty", "path to citizenship"]):
            return "legal_status"
        elif any(term in text for term in ["daca", "dreamer", "child", "family separation"]):
            return "children"
        elif any(term in text for term in ["asylum", "refugee"]):
            return "asylum"
        elif any(term in text for term in ["immigration", "immigrant", "migrant"]):
            return "general"
        else:
            return "other"
    
    df_clean['speech_category'] = df_clean['full_text'].apply(categorize_speech)

# Ensure 'date_standard' column exists (standardized date format)
if 'date' in df_clean.columns and 'date_standard' not in df_clean.columns:
    print("Standardizing dates...")
    
    def standardize_date(date_str):
        try:
            if pd.isna(date_str) or date_str == "Unknown":
                return None
                
            # Handle different date formats
            formats = ["%A, %B %d, %Y", "%Y-%m-%d", "%m/%d/%Y"]
            
            for fmt in formats:
                try:
                    date_obj = pd.to_datetime(date_str, format=fmt)
                    return date_obj.strftime("%Y-%m-%d")
                except:
                    continue
                    
            return date_str
        except:
            return date_str
    
    df_clean['date_standard'] = df_clean['date'].apply(standardize_date)

# Add text excerpt for Excel
if 'full_text' in df_clean.columns:
    print("Creating text excerpts...")
    df_clean['text_excerpt'] = df_clean['full_text'].apply(
        lambda x: (str(x)[:500] + '...') if isinstance(x, str) and len(str(x)) > 500 else x
    )

# Add text length column
if 'full_text' in df_clean.columns:
    df_clean['text_length'] = df_clean['full_text'].apply(
        lambda x: len(str(x)) if not pd.isna(x) else 0
    )

# Create compact version (no full text)
compact_df = df_clean.copy()
if 'full_text' in compact_df.columns:
    compact_df = compact_df.drop('full_text', axis=1)

# Create Excel version (no full text, with excerpt)
excel_df = df_clean.copy()
if 'full_text' in excel_df.columns:
    excel_df = excel_df.drop('full_text', axis=1)

# Save the compact version
compact_csv = os.path.join(output_dir, "immigration_data_compact.csv")
compact_df.to_csv(compact_csv, index=False)

# Save the Excel-friendly version
excel_file = os.path.join(output_dir, "immigration_data_excel.xlsx")
with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
    excel_df.to_excel(writer, sheet_name='Speeches', index=False)
    
    # Add some formatting
    workbook = writer.book
    worksheet = writer.sheets['Speeches']
    
    # Set default column widths
    for col_num, _ in enumerate(excel_df.columns):
        worksheet.set_column(col_num, col_num, 15)
    
    # Set specific columns wider if they exist
    col_widths = {
        'file_id': 30,
        'full_text': 70,
        'text_excerpt': 70,
        'title': 30
    }
    
    for col_name, width in col_widths.items():
        if col_name in excel_df.columns:
            col_idx = excel_df.columns.get_loc(col_name)
            worksheet.set_column(col_idx, col_idx, width)

# Create sample text files
text_dir = os.path.join(output_dir, "speech_samples")
os.makedirs(text_dir, exist_ok=True)

if 'full_text' in df_clean.columns:
    print("Creating sample text files...")
    
    # Get sample of speeches (max 20)
    sample_size = min(20, len(df_clean))
    for idx, row in df_clean.head(sample_size).iterrows():
        # Create a clean filename
        if 'speaker_last' in row and not pd.isna(row['speaker_last']):
            speaker = str(row['speaker_last'])
        else:
            speaker = "Unknown"
            
        clean_name = ''.join(c if c.isalnum() else '_' for c in speaker)
        
        # Add date to filename if available
        date_str = ""
        if 'date_standard' in row and not pd.isna(row['date_standard']):
            date_str = str(row['date_standard'])
        
        file_name = f"{clean_name}_{date_str}_{idx}.txt"
        file_path = os.path.join(text_dir, file_name)
        
        with open(file_path, 'w', encoding='utf-8') as f:
            # Write metadata
            metadata_cols = ['speaker_full', 'speaker_last', 'party', 'date', 'title', 'chamber', 'speech_category']
            for col in metadata_cols:
                if col in row and not pd.isna(row[col]):
                    f.write(f"{col}: {row[col]}\n")
            
            f.write("\n" + "="*50 + "\n\n")
            
            # Write full text
            if 'full_text' in row and not pd.isna(row['full_text']):
                f.write(str(row['full_text']))
            else:
                f.write("[No text available]")
            
print(f"\nCreated Excel-friendly version: {excel_file}")
print(f"Created compact CSV without full text: {compact_csv}")
print(f"Created sample text files in: {text_dir}")

# Print summary statistics
print("\nSummary statistics:")
if 'party' in df_clean.columns:
    party_counts = df_clean['party'].value_counts()
    print("\nSpeeches by party:")
    print(party_counts)

if 'speech_category' in df_clean.columns:
    category_counts = df_clean['speech_category'].value_counts()
    print("\nSpeeches by category:")
    print(category_counts)

Loaded 1094 records from clean speeches CSV
Available columns in the dataset:
- file_id
- date_standard
- chamber
- speaker_full
- speaker_last
- party
- title
- is_speech
- speech_category
- speech_summary
- immigration_terms
- full_text
Creating text excerpts...
Creating sample text files...

Created Excel-friendly version: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data\immigration_data_excel.xlsx
Created compact CSV without full text: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data\immigration_data_compact.csv
Created sample text files in: C:\Users\kevin barcenas\Documents\GitHub\LIN350Project\processed_data\speech_samples

Summary statistics:

Speeches by party:
party
D    147
R     92
I     10
Name: count, dtype: int64

Speeches by category:
speech_category
children           406
other              283
border_security    181
general             92
legal_status        89
asylum              43
Name: count, dtype: int64


Great now going forward I'll have these two files to work with
"processed_data/immigration_data_compact.csv" and "processed_data/immigration_speeches_clean.csv"

In [24]:
full_df = pd.read_csv("processed_data/immigration_speeches_clean.csv")
full_text_df = full_df["full_text"]
print(full_text_df.iloc[0])


[Congressional Record Volume 163, Number 141 (Friday, September 1, 2017)]
[Extensions of Remarks]
[Page E1151]
From the Congressional Record Online through the Government Publishing Office [www.gpo.gov]




 HONORING THE 40TH ANNIVERSARY OF SERVICE BY REVEREND DR. AMOS C. BROWN

                                 ______
                                 

                           HON. NANCY PELOSI

                             of california

                    in the house of representatives

                       Friday, September 1, 2017

  Ms. PELOSI. Mr. Speaker, I rise with pride today to join my beloved 
city in recognizing the 40th Anniversary of Reverend Dr. Amos C. Brown 
as the pastor of the Third Baptist Church of San Francisco and honoring 
his lifetime of moral leadership, political activism and spiritual 
guidance.
  Born and raised in Jackson, Mississippi, Reverend Brown's history of 
activism and his deep involvement in the civil rights movement began at 
an early age